<a href="https://colab.research.google.com/github/fadsdads/thesisv3/blob/main/03_Fit_CE_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#
# CELL 1: Install Dependencies
#
!pip install pymatgen smol pandas tqdm pyngrok mlflow matplotlib scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 77.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [15]:
#
# CELL 2: SETUP AND CE FITTING WORKFLOW
#
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
from tqdm.auto import tqdm
import mlflow
from pyngrok import ngrok

# Pymatgen and smol imports
from pymatgen.core import Structure
from pymatgen.entries.computed_entries import ComputedStructureEntry
# Corrected smol imports for version 0.5.7
from smol.cofe import ClusterSubspace, StructureWrangler, ClusterExpansion
from smol.io import save_work

# Scikit-learn for fitting
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

drive.mount('/content/drive')

#@title Configure CE Fitting { display-mode: "form" }
#@markdown ---
#@markdown Enter the TM Triplet and select the Phase to model.
tm_triplet = "Ni-Mn-Co" #@param {type:"string"}
phase = "O3" #@param ["O3", "P3"]
#@markdown ---

# --- Paths ---
BASE_PROJECT_PATH = "/content/drive/MyDrive/SodiumIonProject"
TRIPLET_PATH = os.path.join(BASE_PROJECT_PATH, tm_triplet)
REPO_PATH = "/content/thesisv3" # Path to your cloned repo in Colab
OUTPUT_DIR = os.path.join(TRIPLET_PATH, "3_CE_Models")
MLFLOW_URI = f"file://{BASE_PROJECT_PATH}/mlflow_data"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# --- MLflow Configuration ---
mlflow.set_tracking_uri(MLFLOW_URI)
mlflow.set_experiment(f"Stability - {tm_triplet}")

#
# --- HELPER FUNCTION: Find Optimal Model (Cutoffs & LASSO Alpha) ---
#
def find_optimal_ce_model(wrangler, basis, tm_triplet, phase):
    """
    Finds the optimal cutoffs and LASSO alpha hyperparameter,
    logs the process, and returns the final fitted ClusterExpansion.
    """
    print("Step 1: Finding optimal cutoffs...")
    pair_cutoffs = np.arange(4, 8, 1.0)
    triplet_cutoffs = np.arange(3, 5, 0.5)

    # Use a fixed, reasonable alpha for the initial cutoff search
    initial_alpha = 1e-3
    estimator = Lasso(alpha=initial_alpha, fit_intercept=False, max_iter=5000)
    cutoff_scores = {}

    for p_cut in tqdm(pair_cutoffs, desc="Testing Cutoffs"):
        for t_cut in triplet_cutoffs:
            if t_cut >= p_cut: continue

            subspace = ClusterSubspace.from_cutoffs(basis, cutoffs={2: p_cut, 3: t_cut})
            wrangler.update_cluster_subspace(subspace)

            scores = cross_val_score(estimator, wrangler.feature_matrix, wrangler.get_property_vector('energy'), scoring='neg_root_mean_squared_error', cv=5)
            cutoff_scores[(p_cut, t_cut)] = -np.mean(scores)

    best_cutoffs, _ = min(cutoff_scores.items(), key=lambda item: item[1])
    optimal_pair_cut, optimal_triplet_cut = best_cutoffs
    print(f"Optimal cutoffs found: Pair={optimal_pair_cut} Å, Triplet={optimal_triplet_cut} Å")
    mlflow.log_params({"optimal_pair_cutoff": optimal_pair_cut, "optimal_triplet_cutoff": optimal_triplet_cut})

    # Now, find the optimal alpha using the best cutoffs
    print("\nStep 2: Finding optimal LASSO alpha...")
    optimal_subspace = ClusterSubspace.from_cutoffs(basis, cutoffs={2: optimal_pair_cut, 3: optimal_triplet_cut})
    wrangler.update_cluster_subspace(optimal_subspace)

    alphas_to_test = np.logspace(-4, -2, 20)
    alpha_scores = {}
    for alpha in tqdm(alphas_to_test, desc="Testing Alphas"):
        estimator = Lasso(alpha=alpha, fit_intercept=False, max_iter=5000)
        scores = cross_val_score(estimator, wrangler.feature_matrix, wrangler.get_property_vector('energy'), scoring='neg_root_mean_squared_error', cv=5)
        alpha_scores[alpha] = -np.mean(scores)

    best_alpha, min_rmse = min(alpha_scores.items(), key=lambda item: item[1])
    print(f"Optimal LASSO alpha found: {best_alpha:.5f} with CV-RMSE: {min_rmse:.4f} eV/prim")
    mlflow.log_param("optimal_lasso_alpha", best_alpha)

    # Create and log the elbow plot for cutoffs
    plt.figure(); plt.plot([k[0] for k,v in cutoff_scores.items() if k[1]==optimal_triplet_cut], [v for k,v in cutoff_scores.items() if k[1]==optimal_triplet_cut], marker='o')
    plt.xlabel("Pair Cutoff (Å)"); plt.ylabel("CV RMSE (eV/prim)"); plt.title(f"Cutoff Elbow Plot - {phase}"); plt.grid(True)
    plot_filename = f"elbow_plot_{tm_triplet}_{phase}.png"; plt.savefig(plot_filename); mlflow.log_artifact(plot_filename); plt.close()

    # Create and log the elbow plot for alpha
    plt.figure(); plt.semilogx(list(alpha_scores.keys()), list(alpha_scores.values()), marker='o')
    plt.xlabel("LASSO Alpha"); plt.ylabel("CV RMSE (eV/prim)"); plt.title(f"LASSO Alpha Elbow Plot - {phase}"); plt.grid(True)
    alpha_plot_filename = f"alpha_elbow_plot_{tm_triplet}_{phase}.png"; plt.savefig(alpha_plot_filename); mlflow.log_artifact(alpha_plot_filename); plt.close()


    # Fit final model with optimal settings
    final_estimator = Lasso(alpha=best_alpha, fit_intercept=False, max_iter=5000)
    final_estimator.fit(wrangler.feature_matrix, wrangler.get_property_vector('energy'))

    final_expansion = ClusterExpansion(optimal_subspace, coefficients=final_estimator.coef_)
    return final_expansion

#
# --- Main CE Fitting Workflow ---
#
run_name = f"CE-Fit-LASSO_{tm_triplet}_{phase}"
with mlflow.start_run(run_name=run_name):
    print(f"\n--- Starting CE Fit for {tm_triplet} - {phase} phase ---")
    mlflow.log_params({"phase": phase, "tm_triplet": tm_triplet})
    mlflow.set_tag("step", "03_Fit_CE_Models")

    relaxed_dir = os.path.join(TRIPLET_PATH, phase, "2_Relaxed_Structures")
    log_file = os.path.join(relaxed_dir, "optimization_log.csv")

    if not os.path.exists(log_file):
        raise FileNotFoundError(f"ERROR: Log file not found for {phase}. Please run Notebook 02 first.")

    energy_df = pd.read_csv(log_file)
    entries = [ComputedStructureEntry(Structure.from_file(os.path.join(relaxed_dir, row['Filename'])), row['Energy_eV']) for _, row in energy_df.iterrows()]
    print(f"Loaded {len(entries)} relaxed structures.")

    # Define Basis Set and create Wrangler
    base_cif_path = os.path.join(REPO_PATH, "NaCoO2_for_substitution.cif")
    # In smol v0.5.7, the prim is just a pymatgen Structure object.
    prim = Structure.from_file(base_cif_path)
    wrangler = StructureWrangler()
    for entry in entries:
        wrangler.add_entry(entry, verbose=False)

    # Find optimal parameters and fit the final model
    fitted_expansion = find_optimal_ce_model(wrangler, prim, tm_triplet, phase)

    # Log final metrics
    final_predictions = fitted_expansion.predict(wrangler.structures)
    final_rmse = mean_squared_error(wrangler.get_property_vector('energy'), final_predictions, squared=False)
    mlflow.log_metric("final_rmse", final_rmse)
    print(f"\nFinal Model RMSE: {final_rmse:.4f} eV/prim")

    # Save Model
    model_filename = f"ce_model_lasso_{tm_triplet}_{phase}.json"
    model_path = os.path.join(OUTPUT_DIR, model_filename)
    save_work(model_path, fitted_expansion)
    mlflow.log_artifact(model_path)

    print(f"--- Fit complete. Model saved to {model_path} ---")

#
# --- VIEW MLFLOW RESULTS ---
#
ngrok.kill()
get_ipython().system_raw(f"mlflow ui --backend-store-uri {MLFLOW_URI} &")
public_url = ngrok.connect(5000)
print("\nClick the MLflow UI link below to see your results:")
print(public_url)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

--- Starting CE Fit for Ni-Mn-Co - O3 phase ---
Loaded 6 relaxed structures.


/usr/local/lib/python3.11/dist-packages/pymatgen/io/cif.py:1313: UserWarning: Missing elements Na from PMG structure composition
  if struct := self._get_structure(data, primitive, symmetrized, check_occu=check_occu):


FileNotFoundError: [Errno 2] No such file or directory: '/content/thesisv3/NaCoO2_for_substitution.cif'